In [1]:
from train_contrastive import load_data

In [ ]:
import jax


In [2]:
path = "../data/tabula_muris/preprocessed/tm_adata_test.pkl"
path_r = "../data/tabula_muris/preprocessed_reduced/tm_adata_test.pkl"

X, domains, cells, adata = load_data(path)
X_r, domains_r, cells_r, adata_r = load_data(path_r)

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
from contrastive_model import ContrastiveModel

CONTRASTIVE_TRANSFORMER_KEY="contrastive_transformer"
CONTRASTIVE_MLP_KEY="contrastive_mlp"

c_transformer_path = "runs/contrastive_attention_d256_h2_l2_t1.0_05-08-23:08/final.ckpt"
c_mlp_path = "runs/contrastive_dense_d512_l12_t1.0_05-08-21:16/final.ckpt"


c_transformer = ContrastiveModel.load_from_checkpoint(c_transformer_path)
c_mlp = ContrastiveModel.load_from_checkpoint(c_mlp_path)

In [5]:
cuda_device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Set tensors to the appropriate device
x = torch.tensor(X, dtype=torch.float32).to(cuda_device)
x_r = torch.tensor(X_r, dtype=torch.float32).to(cuda_device)

c_transformer.to(cuda_device)
c_mlp.to(cuda_device)


ContrastiveModel(
  (encoder): DenseEncoder(
    (encoder): Sequential(
      (0): Linear(in_features=2000, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): LeakyReLU(negative_slope=0.01)
      (6): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=512, out_features=512, bias=True)
      (9): LeakyReLU(negative_slope=0.01)
      (10): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (11): Dropout(p=0.1, inplace=False)
      (12): Linear(in_features=512, out_features=512, bias=True)
      (13): LeakyReLU(negative_slope=0.01)
      (14): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (15): Dropout(p=0.1, inplace=False)
      (16): Linear(in_features=512, out_features=512, bias=True)
      (1

In [6]:
import torch
from tqdm import tqdm

batch_size = 32  # Define a suitable batch size
ct_embeddings_list = []
cm_embeddings_list = []

c_transformer.eval()
c_mlp.eval()

with torch.no_grad():
    for i in tqdm(range(0, x.size(0), batch_size), desc="Processing Batches"):
        x_batch = x[i:i + batch_size]
        x_r_batch = x_r[i:i + batch_size]
        
        ct_embeddings_batch = c_transformer.encoder(x_r_batch)
        cm_embeddings_batch = c_mlp.encoder(x_batch)
        
        ct_embeddings_list.append(ct_embeddings_batch)
        cm_embeddings_list.append(cm_embeddings_batch)

ct_embeddings = torch.cat(ct_embeddings_list, dim=0)
cm_embeddings = torch.cat(cm_embeddings_list, dim=0)

Processing Batches: 100%|████████████████████████████████████| 1931/1931 [00:09<00:00, 210.83it/s]


In [7]:
adata.obsm[CONTRASTIVE_TRANSFORMER_KEY] = ct_embeddings.cpu().numpy()
adata.obsm[CONTRASTIVE_MLP_KEY] = cm_embeddings.cpu().numpy()

/tmp/ipykernel_998471/491995230.py:1: ImplicitModificationWarning: Setting element `.obsm['contrastive_transformer']` of view, initializing view as actual.
  adata.obsm[CONTRASTIVE_TRANSFORMER_KEY] = ct_embeddings.cpu().numpy()


In [8]:
import scanpy as sc

In [9]:
sc.pp.neighbors(adata, use_rep=CONTRASTIVE_TRANSFORMER_KEY)  # Compute neighbors using latent space
sc.tl.umap(adata)
fig1 = sc.pl.umap(adata, color='tech', legend_loc=None, title='', frameon=False, size=50, return_fig=True, show=True)
fig1.savefig('umap_transformer.png')

/oscar/rt/9.2/software/0.20-generic/0.20.1/opt/spack/linux-rhel9-x86_64_v3/gcc-11.3.1/anaconda-2023.09-0-7nso27ys7navjquejqdxqylhg7kuyvxo/lib/python3.11/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [10]:
sc.pp.neighbors(adata, use_rep=CONTRASTIVE_MLP_KEY)  # Compute neighbors using latent space
sc.tl.umap(adata)
fig = sc.pl.umap(adata, color='tech', legend_loc=None, title='', frameon=False, size=50, return_fig=True, show=True)
fig.savefig('umap_mlp.png')

In [ ]:
from scib_metrics.benchmark import Benchmarker
bm = Benchmarker(
    adata,
    batch_key='tech',
    label_key='cell_ontology_class',
    embedding_obsm_keys=[CONTRASTIVE_TRANSFORMER_KEY, CONTRASTIVE_MLP_KEY],
    n_jobs=-1
)
bm.benchmark()

/users/ndepiero/.local/lib/python3.11/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
Embeddings:   0%|                                                           | 0/2 [00:00<?, ?it/s]WARNING:jax._src.xla_bridge:An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
